In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
sub_rowdata = pd.read_csv(r'datasets/@liuf109#3ac05fd206366fe07ad4a1b0a2167d78/data_v1_train.csv')
#print(sub_rowdata.head(3))#可成功读到数据
#print(sub_rowdata['ID'].head(3))
# sub_rowdata = pd.DataFrame(sub_rowdata)

split_rowdata = sub_rowdata['time'].astype('str').str.split(':',expand = True)
split_rowdata.columns = ['Time','notrequire1']

sub_rowdata = pd.concat([sub_rowdata['ID'],sub_rowdata['flow'],sub_rowdata['time'],split_rowdata['Time']],axis = 1)
sub_rowdata['Time'] = pd.to_datetime(sub_rowdata['Time'])
sub_rowdata.set_index('Time')
#print(sub_rowdata)
sub_rowdata_group = pd.DataFrame(sub_rowdata['flow'].groupby([sub_rowdata['ID'],sub_rowdata['Time']]).max())
sub_rowdata_group.reset_index(inplace = True)
#print(sub_rowdata_group)

list = []
test_network_ids = sub_rowdata_group['ID'].unique().tolist()
#for index,raw in sub_rowdata_group.iterrows():
#    sub_rowdata_group['ID'][index] = test_network_ids.index(raw['ID'])
#class_mapping = {'A':0, 'B':1}
#data[class] = data[class].map(class_mapping)
shunxu = []
for i in range(1,73):
    shunxu.append(i)
dictshunxu = {test_network_ids[i]:shunxu[i] for i in range(len(test_network_ids))}
#print(dictshunxu)
sub_rowdata_group['ID'] = sub_rowdata_group['ID'].map(dictshunxu)
#sub_rowdata_group['ID'] = test_network_ids.index(sub_rowdata_group['ID'])
print(sub_rowdata_group)
sub_rowdata_group.to_csv(r'datasets/@liuf109#3ac05fd206366fe07ad4a1b0a2167d78/0LSTM/rowdata4/data_v1_train.csv',index = None)

        ID                Time      flow
0        1 2021-07-30 12:00:00  0.000000
1        1 2021-07-30 13:00:00  0.000000
2        1 2021-07-30 14:00:00  0.000000
3        1 2021-07-30 15:00:00  0.000000
4        1 2021-07-30 16:00:00  0.000035
...     ..                 ...       ...
399042  72 2022-01-05 19:00:00  4.355480
399043  72 2022-01-05 20:00:00  5.709647
399044  72 2022-01-05 21:00:00  5.444348
399045  72 2022-01-05 22:00:00  3.633421
399046  72 2022-01-05 23:00:00  3.885509

[399047 rows x 3 columns]


In [2]:
train_hours = sub_rowdata_group[['Time','ID','flow']]
#train_hours = train_hours.sort_values('Time').groupby(['ID'], as_index=False)
#train_hours = train_hours.agg({'flow'})
#train_hours.columns = ['Time', 'ID', 'flow']
train_hours = train_hours.query('Time >= "2021-11-11 00:00:00" and Time <= "2022-01-05 00:00:00"')
print(train_hours.head())
train_hours['flow_cnt_hours'] = train_hours.sort_values('Time').groupby(['ID'])['flow'].shift(-1)
print(train_hours.head())

monthly_series = train_hours.pivot_table(index=['ID'], columns='Time',values='flow', fill_value=0).reset_index()
monthly_series.head()



                    Time  ID       flow
2484 2021-11-11 00:00:00   1  14.940597
2485 2021-11-11 01:00:00   1  11.228764
2486 2021-11-11 02:00:00   1   7.789449
2487 2021-11-11 03:00:00   1   5.606784
2488 2021-11-11 04:00:00   1   4.346419
                    Time  ID       flow  flow_cnt_hours
2484 2021-11-11 00:00:00   1  14.940597       11.228764
2485 2021-11-11 01:00:00   1  11.228764        7.789449
2486 2021-11-11 02:00:00   1   7.789449        5.606784
2487 2021-11-11 03:00:00   1   5.606784        4.346419
2488 2021-11-11 04:00:00   1   4.346419        2.765237


Time,ID,2021-11-11 00:00:00,2021-11-11 01:00:00,2021-11-11 02:00:00,2021-11-11 03:00:00,2021-11-11 04:00:00,2021-11-11 05:00:00,2021-11-11 06:00:00,2021-11-11 07:00:00,2021-11-11 08:00:00,...,2022-01-04 15:00:00,2022-01-04 16:00:00,2022-01-04 17:00:00,2022-01-04 18:00:00,2022-01-04 19:00:00,2022-01-04 20:00:00,2022-01-04 21:00:00,2022-01-04 22:00:00,2022-01-04 23:00:00,2022-01-05 00:00:00
0,1,14.940597,11.228764,7.789449,5.606784,4.346419,2.765237,2.164130,1.909579,1.764144,...,32.276536,31.344990,38.202700,32.211554,32.570896,32.464790,33.916440,34.193040,41.806024,33.298078
1,2,171.393376,194.398368,130.993616,99.554840,80.535488,92.845264,14.909455,11.569733,39.393544,...,72.541768,66.852568,49.098912,39.061472,20.710768,36.346252,22.144236,14.566312,13.964399,145.212896
2,3,1.337608,0.898661,0.892589,0.950358,0.971776,1.031840,1.023965,3.603804,11.632146,...,16.385352,26.629262,21.052872,20.065878,19.814414,10.579760,2.125028,1.222016,0.831786,0.876630
3,4,13.628420,13.348012,12.889409,43.200124,20.730514,14.674860,15.516320,14.457233,51.089156,...,10.914105,34.225372,12.173839,11.977692,12.526954,17.508036,11.938904,12.032740,12.688840,10.493511
4,5,98.822280,98.797248,100.314008,99.969200,100.047800,100.329056,101.291632,99.652504,100.145744,...,58.850700,58.355488,57.270252,52.769920,48.739160,46.607900,44.257068,40.258384,37.989864,34.217884


In [3]:
data_series = pd.DataFrame(monthly_series)
test_network_ids = data_series['ID'].unique()
#print(data_series.head())
#print(data_series.shape)
from sklearn.model_selection import train_test_split
# 最后N条数据作为测试数据
testNum = 1
# 将数据分割为训练集和测试集，此时分割的数据集是二维数组（取最后12条数据作为测试数据）
train, test = data_series.iloc[:,:-testNum], monthly_series.iloc[:,-testNum:]
print(train.shape)
print(test.shape)
train, valid, Y_train, Y_valid = train_test_split(train, test.values, test_size=0.10, random_state=0)

#print(train)
#print(valid)
X_train = train.values.reshape((train.shape[0], train.shape[1], 1))
X_valid = valid.values.reshape((valid.shape[0], valid.shape[1], 1))

print("Train set reshaped", X_train.shape)
print("Validation set reshaped", X_valid.shape)

(72, 1321)
(72, 1)
Train set reshaped (64, 1321, 1)
Validation set reshaped (8, 1321, 1)


In [4]:
from tensorflow.keras import optimizers, Sequential, Model
import tensorflow.keras.layers as L

serie_size =  X_train.shape[1] # 12
n_features =  X_train.shape[2] # 1

epochs = 10
batch = 128
lr = 0.0001

lstm_model = Sequential()
lstm_model.add(L.LSTM(10, input_shape=(serie_size, n_features), return_sequences=True))
lstm_model.add(L.LSTM(6, activation='relu', return_sequences=True))
lstm_model.add(L.LSTM(1, activation='relu'))
lstm_model.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(L.Dense(10, kernel_initializer='glorot_normal', activation='relu'))
lstm_model.add(L.Dense(1))
lstm_model.summary()

adam = optimizers.Adam(lr)
lstm_model.compile(loss='mean_absolute_error', optimizer='adam',metrics = ['mae'])


lstm_history = lstm_model.fit(X_train, Y_train,
                              validation_data=(X_valid, Y_valid),
                              batch_size=batch,
                              epochs=epochs,
                              verbose=2)

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1321, 10)          480       
_________________________________________________________________
lstm_1 (LSTM)                (None, 1321, 6)           408       
_________________________________________________________________
lstm_2 (LSTM)                (None, 1)                 32        
_________________________________________________________________
dense (Dense)                (None, 10)                20        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 1,061
Trainable params: 1,061
Non-trainable par

In [5]:
from sklearn.metrics import mean_absolute_error
print(X_valid.shape)
lstm_train_pred = lstm_model.predict(X_train)
lstm_val_pred = lstm_model.predict(X_valid)
print(lstm_val_pred.shape)
print('Train mae:', mean_absolute_error(Y_train, lstm_train_pred))
print('Validation mae:', mean_absolute_error(Y_valid, lstm_val_pred))

(8, 1321, 1)
(8, 1)
Train mae: 33.865762603935536
Validation mae: 56.376845669468395


In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Dropout
import tensorflow.keras.layers as L
from tensorflow.keras import optimizers, Sequential, Model

# our defining our model 
my_model = Sequential()
my_model.add(LSTM(units = 72,input_shape = (1321,1)))
my_model.add(Dropout(0.4))
my_model.add(Dense(1))

my_model.compile(loss = 'mse',optimizer = 'adam', metrics = ['mean_absolute_error'])
my_model.summary()
my_model.fit(X_train,Y_train,batch_size = 4096,epochs = 20)

# creating submission file 
lstm_train_pred = lstm_model.predict(X_train)
submission_pfs = my_model.predict(X_valid)
print('Train mae:', mean_absolute_error(Y_train, lstm_train_pred))
print('Validation mae:', mean_absolute_error(Y_valid, lstm_val_pred))
# we will keep every value between 0 and 20
#submission_pfs = submission_pfs.clip(0,20)
# creating dataframe with required columns 
#submission = pd.DataFrame({'ID':test_data['ID'],'item_cnt_month':submission_pfs.ravel()})
# creating csv file from dataframe
#submission.to_csv('sub_pfs.csv',index = False)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_11 (LSTM)               (None, 72)                21312     
_________________________________________________________________
dropout_8 (Dropout)          (None, 72)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 24)                1752      
Total params: 23,064
Trainable params: 23,064
Non-trainable params: 0
_________________________________________________________________


ValueError: Error when checking target: expected dense_11 to have shape (24,) but got array with shape (1,)

In [ ]:
last_month = serie_size - 1
Y_train_encoded = train['label']
train.drop('label', axis=1, inplace=True)
X_train_encoded = train[[last_month, 'encoded']]

Y_valid_encoded = valid['label']
valid.drop('label', axis=1, inplace=True)
X_valid_encoded = valid[[last_month, 'encoded']]

print("Train set", X_train_encoded.shape)
print("Validation set", X_valid_encoded.shape)